<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>


<br>

# <font color="#76b900">**Notebook 3:** LangChain 表達式語言(Expression Language)</font>

<br>

在上一個 notebook 中，我們介紹了一些我們將在 LLM 應用程式中利用的服務，包括一些外部 LLM 平台和本地託管(Hosting)的前端服務。這兩個組件都整合了 LangChain，但它還沒有被突出作為主要焦點。預期您對 LangChain 和 LLM 有一些經驗，但這個 notebook 旨在讓您跟上進度，為後續章節做準備！

這個 notebook 旨在引導您整合和應用 LangChain，這是一個領先的大型語言模型 (LLM) 流程協調管理(Orchestration)函式庫(library)，與上次的 AI 基礎模型端點(Endpoints)結合使用。無論您是經驗豐富的開發者還是 LLM 新手，這門課程都將增強您在建構複雜 LLM 應用程式方面的理解和技能。

<br>

### **學習目標：**


-   學習如何利用鏈(Chain)和可運行物件來流程協調管理(Orchestration)有趣的 LLM 系統。

-   熟悉使用 LLM 進行外部對話和內部分析推理(Reasoning)。

-   能夠在您的 notebook 中啟動並運行簡單的 [**Gradio**](https://www.gradio.app/) 介面。


<br>

### **值得思考的問題：**

-   什麼樣的實用工具可能是保持資訊在管線(Pipeline)中流動所必需的 **（下一個 notebook 的入門）**。
-   當您遇到 [**Gradio**](https://www.gradio.app/) 時，考慮您之前在哪些地方見過這種風格的介面。一些可能的地方可能包括 [**HuggingFace Spaces**](https://huggingface.co/spaces)...
-   在章節末尾，您將了解到您可以將鏈(Chain)作為路由(Routes)傳遞，並透過連接埠跨環境存取它們。如果您試圖從其他微服務(MICROSERVICES)接收鏈(Chain)，您應該廣告什麼樣的要求？

<br>

### **環境設置：**






In [1]:
## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio

# import os
# os.environ["NVIDIA_API_KEY"] = "nvapi-..."

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

<br>

### **考慮您的模型**


回到 [**NVIDIA NGC 目錄**](https://catalog.ngc.nvidia.com/)，我們將能夠找到一些有趣的模型選擇，您可以從您的環境中invoke。這些模型都在那裡是因為它們在正式環境(Production)管線(Pipeline)中有有效用途，所以四處看看並找出哪些模型最適合您的使用案例是個好主意。

**提供的程式碼已經包含了一些已列出的模型，但如果您注意到有嚴格更好的選項或某個模型不再可用，您可能想要（或需要）升級到其他模型。*這個註解將適用於課程的其餘部分，所以請記住這一點！***




<br>

## **第一部分：** 什麼是 LangChain？







LangChain 是一個常見(popular)的 LLM 流程協調管理(Orchestration)函式庫(library)，用於幫助設置具有一個或多個 LLM 組件的系統。這個函式庫(library)，無論好壞，都極其常見(popular)，並且基於該領域的新發展而快速變化，這意味著某人可能在 LangChain 的某些部分有很多經驗，而對其他部分幾乎沒有或完全沒有熟悉度（要麼是因為有太多不同的功能，要麼是該領域是新的，功能只是最近才實作）。

這個 notebook 將使用 **LangChain 表達式語言 (LCEL)** 從基本鏈(Chain)規範提升到更高級的對話管理實踐，所以希望這個旅程會很愉快，甚至經驗豐富的 LangChain 開發者也可能學到一些新東西！

<!-- > <img style="max-width: 400px;" src="imgs/langchain-diagram.png" /> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/langchain-diagram.png" width=400px/>
<!-- > <img src="https://drive.google.com/uc?export=view&id=1NS7dmLf5ql04o5CyPZnd1gnXXgO8-jbR" width=400px/> -->




<br>

## **第二部分：** 鏈(Chain)和 Runnables


在探索新函式庫(library)時，重要的是要注意函式庫(library)的核心系統是什麼以及它們是如何使用的。

在 LangChain 中，主要的構建塊*過去是*經典的 **鏈(Chain)**：一個執行特定功能的小模組，可以與其他鏈(Chain)連結起來組成一個系統。因此，就所有意圖和目的而言，它是一個「構建塊系統(building-block system)」抽象化(abstractions)，其中構建塊(building blocks)易於創建，具有一致的方法（`invoke`、`generate`、`stream` 等），並且可以連結起來作為一個系統一起工作。一些傳統鏈(Chain)的範例包括 `LLMChain`、`ConversationChain`、`TransformationChain`、`SequentialChain` 等。

最近，出現了一個新的推薦規範，它顯著更容易使用且極其簡潔(compact)，即 **LangChain 表達式語言 (LCEL)**。這種新格式依賴於不同類型的原語 - **Runnable** - 它只是一個包裝函式(function)的物件。允許字典(dictionaries)隱式轉換(implicitly converted)為Runnables，並讓 **pipe |** 運算子創建一個將資料從左傳遞到右的Runnable（即 `fn1 | fn2` 是一個Runnable），您就有了一種指定複雜邏輯的簡單方法！

以下是一些非常具代表性的範例Runnable，透過 `RunnableLambda` 類別創建：

In [2]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

################################################################################
## Very simple "take input and return it"
identity = RunnableLambda(lambda x: x)  ## Or RunnablePassthrough works

################################################################################
## Given an arbitrary function, you can make a runnable with it
def print_and_return(x, preface=""):
    print(f"{preface}{x}")
    return x

rprint0 = RunnableLambda(print_and_return)

################################################################################
## You can also pre-fill some of values using functools.partial
rprint1 = RunnableLambda(partial(print_and_return, preface="1: "))

################################################################################
## And you can use the same idea to make your own custom Runnable generator
def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

################################################################################
## Chaining two runnables
chain1 = identity | rprint0
chain1.invoke("Hello World!")
print()

################################################################################
## Chaining that one in as well
output = (
    chain1           ## Prints "Welcome Home!" & passes "Welcome Home!" onward
    | rprint1        ## Prints "1: Welcome Home!" & passes "Welcome Home!" onward
    | RPrint("2: ")  ## Prints "2: Welcome Home!" & passes "Welcome Home!" onward
).invoke("Welcome Home!")

## Final Output Is Preserved As "Welcome Home!"
print("\nOutput:", output)

Hello World!

Welcome Home!
1: Welcome Home!
2: Welcome Home!

Output: Welcome Home!



<br>

## **第三部分：** 運用聊天模型(Chat Models)及字典管線(Dictionary Pipeline)


您可以用Runnable做很多事情，但重要的是正式化(Formalize)一些最佳實踐。目前，使用*dictionaries*作為我們的預設變數容器是最容易的，原因如下：

**傳遞dictionaries幫助我們按名稱追蹤變數。**

由於dictionaries允許我們傳播命名變數（由鍵引用的值），使用它們對於鎖定我們鏈(Chain)組件的輸出和期望非常有用。

**LangChain 提示(Prompt)預期收到的dictionary類別的值。**

在 LCEL 中指定一個接受dictionary並產生字串的 LLM 鏈(Chain)是非常直觀的，將該字串提升回dictionary也同樣容易。這是非常有意的，部分是由於上述原因。

<br>

### **範例 1：** 簡單的 LLM 鏈(Chain)


經典 LangChain 最基本的組件之一是接受**提示(Prompt)*\*和 **LLM** 的 `LLMChain`：

-   提示(Prompt)，通常從像 `PromptTemplate.from_template("string with {key1} and {key2}")` 這樣的呼叫中檢索(Retrieval)，指定創建字串作為輸出的模板。可以傳入字典 `{"key1" : 1, "key2" : 2}` 來獲得輸出 `"string with 1 and 2"`。

    -   對於像 `ChatNVIDIA` 這樣的聊天模型，您會使用 `ChatPromptTemplate.from_messages` 代替。

-   LLM 接受字串並返回生成的字串。

    -   像 `ChatNVIDIA` 這樣的聊天模型使用訊息代替，但想法是一樣的！在最後使用 **StrOutputParser** 將從訊息中提取內容。

以下是如上所述的簡單聊天鏈(Chain)的輕量級範例。它所做的就是接受輸入(Intake)字典並使用它填入系統訊息以指定整體元目標和使用者輸入(Intake)以指定查詢模型。

In [3]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Simple Chat Pipeline
chat_llm = ChatNVIDIA(model="meta/llama3-8b-instruct")

prompt = ChatPromptTemplate.from_messages([
    ("system", "Only respond in rhymes"),
    ("user", "{input}")
])

rhyme_chain = prompt | chat_llm | StrOutputParser()

print(rhyme_chain.invoke({"input" : "Tell me about birds!"}))

Birds are quite a delightful find,
With feathers and wings, they soar and entwine.
In trees, they alight, with tails so bright,
And sing their songs, with morning light.

Some have beaks that curve, some have beaks that straight,
Their chirps and chatter, fill the air and create
A symphony sweet, of melodic sound,
As birds take flight, their magic's all around.

From robins to sparrows, to hawks on high,
Each species unique, yet all touch the sky.
With colors bright, and forms so grand,
Birds are a wonder, in this world so bland.


<br>


除了按原樣使用程式碼命令外，我們還可以嘗試使用 [**Gradio 介面**](https://www.gradio.app/guides/creating-a-chatbot-fast)來練習我們的模型。Gradio 是一個常見(popular)的工具，提供簡單的構建塊來創建自訂生成式 AI 介面！下面的範例顯示了如何使用這個特定的範例鏈(Chain)製作一個簡單的 gradio 聊天介面：

In [4]:
import gradio as gr

#######################################################
## Non-streaming Interface like that shown above

# def rhyme_chat(message, history):
#     return rhyme_chain.invoke({"input" : message})

# gr.ChatInterface(rhyme_chat).launch()

#######################################################
## Streaming Interface

def rhyme_chat_stream(message, history):
    ## This is a generator function, where each call will yield the next entry
    buffer = ""
    for token in rhyme_chain.stream({"input" : message}):
        buffer += token
        yield buffer

## Uncomment when you're ready to try this.
demo = gr.ChatInterface(rhyme_chat_stream).queue()
window_kwargs = {} # or {"server_name": "0.0.0.0", "root_path": "/7860/"}
demo.launch(share=True, debug=True, **window_kwargs) 

## IMPORTANT!! When you're done, please click the Square button (twice to be safe) to stop the session.

/usr/local/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://8cde11603d619413dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8cde11603d619413dd.gradio.live


<br>

### **範例 2：內部回應**


有時，您還希望在回應實際向使用者輸出之前，在幕後進行一些快速分析推理(Reasoning)。執行此任務時，您需要一個內建強大指令跟隨(instruction-following)能力的的模型。

以下是一個「零樣本(Zero-Shot)分類」管線(Pipeline)的範例，它將嘗試將句子分類為幾個類別中的一個。

**依照順序，這個零樣本(Zero-Shot)分類鏈(Chain)會：**
-   接受具有兩個必需鍵的字典，`input` 和 `options`。
-   將其通過零樣本(Zero-Shot)提示(Prompt)傳遞以獲得我們 LLM 的輸入(Intake)。
-   將該字串傳遞給模型以獲得結果。

**任務：** 挑選幾個您認為適合這種任務的模型，看看它們表現如何！具體來說：

-   **嘗試找到在多個範例中達到預期結果的模型。** 如果格式總是易於解析且極其可預測，那麼模型可能是可以的。
-   **嘗試找到速度快的模型！** 這很重要，因為內部分析推理(Reasoning)通常在生成「面向使用者(user-facing)」的回應開始之前在幕後發生。因此，它可能會延遲「面向使用者」生成的開始，使您的系統感覺遲緩。




In [5]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## Feel free to try out some more models and see if there are better lightweight options
## https://build.nvidia.com
instruct_llm = ChatNVIDIA(model="mistralai/mistral-7b-instruct-v0.2")

sys_msg = (
    "Choose the most likely topic classification given the sentence as context."
    " Only one word, no explanation.\n[Options : {options}]"
)

## One-shot classification prompt with heavy format assumptions.
zsc_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_msg),
    ("user", "[[The sea is awesome]]"),
    ("assistant", "boat"),
    ("user", "[[{input}]]"),
])

## Roughly equivalent as above for <s>[INST]instruction[/INST]response</s> format
zsc_prompt = ChatPromptTemplate.from_template(
    f"{sys_msg}\n\n"
    "[[The sea is awesome]][/INST]boat</s><s>[INST]"
    "[[{input}]]"
)

zsc_chain = zsc_prompt | instruct_llm | StrOutputParser()

def zsc_call(input, options=["car", "boat", "airplane", "bike"]):
    return zsc_chain.invoke({"input" : input, "options" : options}).split()[0]

print("-" * 80)
print(zsc_call("Should I take the next exit, or keep going to the next one?"))

print("-" * 80)
print(zsc_call("I get seasick, so I think I'll pass on the trip"))

print("-" * 80)
print(zsc_call("I'm scared of heights, so flying probably isn't for me"))

--------------------------------------------------------------------------------
car
--------------------------------------------------------------------------------
boat
--------------------------------------------------------------------------------
airplane


<br>

### **範例 3：多組件鏈(Multi-Component Chain)**

上一個範例顯示了我們如何透過將字典傳遞到 `提示(Prompt) -> LLM` 鏈(Chain)來強制將字典轉換為字串，所以這是一個容易激發容器選擇的結構(motivate the container choice)。但是將字串輸出轉換回字典同樣容易嗎？

**是的，確實如此！** 最簡單的方法實際上是使用 LCEL *"implicit runnable"* 語法，它允許您使用函式(function)字典（包括鏈(Chain)）作為runnable，該runnable運行每個函式(function)並將值映射到輸出字典中的鍵。

以下是一個範例可以練習這些實用工具，同時還提供一些您在實作中可能會發現有用的額外工具。


In [6]:
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from functools import partial

################################################################################
## Example of dictionary enforcement methods
def make_dictionary(v, key):
    if isinstance(v, dict):
        return v
    return {key : v}

def RInput(key='input'):
    '''Coercing method to mold a value (i.e. string) to in-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

def ROutput(key='output'):
    '''Coercing method to mold a value (i.e. string) to out-like dict'''
    return RunnableLambda(partial(make_dictionary, key=key))

def RPrint(preface=""):
    return RunnableLambda(partial(print_and_return, preface=preface))

################################################################################
## Common LCEL utility for pulling values from dictionaries
from operator import itemgetter

up_and_down = (
    RPrint("A: ")
    ## Custom ensure-dictionary process
    | RInput()
    | RPrint("B: ")
    ## Pull-values-from-dictionary utility
    | itemgetter("input")
    | RPrint("C: ")
    ## Anything-in Dictionary-out implicit map
    | {
        'word1' : (lambda x : x.split()[0]),
        'word2' : (lambda x : x.split()[1]),
        'words' : (lambda x: x),  ## <- == to RunnablePassthrough()
    }
    | RPrint("D: ")
    | itemgetter("word1")
    | RPrint("E: ")
    ## Anything-in anything-out lambda application
    | RunnableLambda(lambda x: x.upper())
    | RPrint("F: ")
    ## Custom ensure-dictionary process
    | ROutput()
)

up_and_down.invoke({"input" : "Hello World"})

A: {'input': 'Hello World'}
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}

In [7]:
## NOTE how the dictionary enforcement methods make it easy to make the following syntax equivalent
up_and_down.invoke("Hello World")

A: Hello World
B: {'input': 'Hello World'}
C: Hello World
D: {'word1': 'Hello', 'word2': 'World', 'words': 'Hello World'}
E: Hello
F: HELLO


{'output': 'HELLO'}



<br>

## **第四部分：[練習]** 重新主題化聊天機器人


下面是一個詩歌生成範例，展示了您如何在單一 Agent 的組織兩個不同的任務。系統回到簡單的 Gradio 範例，但用一些樣板回應和幕後邏輯擴展了它。

它的主要功能如下：

-   在第一次回應時，它將基於您的回應生成一首詩。
-   在後續回應中，它將保持您原始韻律的格式和結構，同時修改詩歌的主題。

**問題：** 目前，系統在第一部分應該運作得很好，但第二部分尚未實作。

**目標：** 實作 `rhyme_chat2_stream` 方法的其餘部分，使 Agent 能夠正常運作。

為了使 gradio 組件更容易分析推理(Reasoning)，提供了一個簡化的 `queue_fake_streaming_gradio` 方法，它將使用標準 Python `input` 方法模擬 gradio 聊天事件循環

In [8]:
[model for model in ChatNVIDIA.get_available_models() 
     if ("mistral" in model.id or "meta/llama" in model.id) 
         and model.model_type in ('chat', None)]

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[Model(id='meta/llama-3.1-405b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.1-8b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.2-1b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=False, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.2-3b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=None, aliases=None, supports_tools=True, supports_structured_output=True, base_model=None),
 Model(id='meta/llama-3.3-70b-instruct', model_type='chat', client='ChatNVIDIA', endpoint=N

In [12]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from copy import deepcopy

instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")  ## Feel free to change the models

prompt1 = ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only respond in rhymes (under 5 sentences and using plain English)"
    "\n\nPROMPT: {input}"
))])

prompt2 =  ChatPromptTemplate.from_messages([("user", (
    "INSTRUCTION: Only responding in rhyme, change the topic of the input poem to be about {topic}!"
    " Make it happy! Try to keep the same sentence structure, but make sure it's easy to recite!"
    " Try not to rhyme a word with itself."
    "\n\nOriginal Poem: {input}"
    "\n\nNew Topic: {topic}"
))])

## These are the main chains, constructed here as modules of functionality.
chain1 = prompt1 | instruct_llm | StrOutputParser()  ## only expects input
chain2 = prompt2 | instruct_llm | StrOutputParser()  ## expects both input and topic
################################################################################
## SUMMARY OF TASK: chain1 currently gets invoked for the first input.
##  Please invoke chain2 for subsequent invocations.

def rhyme_chat2_stream(message, history, return_buffer=True):
    '''This is a generator function, where each call will yield the next entry'''

    first_poem = None
    for entry in history:
        if entry[0] and entry[1]:
            ## If a generation occurred as a direct result of a user input,
            ##  keep that response (the first poem generated) and break out
            first_poem = entry[1]
            break

    if first_poem is None:
        ## First Case: There is no initial poem generated. Better make one up!

        buffer = "Oh! I can make a wonderful poem about that! Let me think!\n\n"
        yield buffer

        ## iterate over stream generator for first generation
        inst_out = ""
        chat_gen = chain1.stream({"input" : message})
        for token in chat_gen:
            inst_out += token
            buffer += token
            yield buffer if return_buffer else token

        passage = "\n\nNow let me rewrite it with a different focus! What should the new focus be?"
        buffer += passage
        yield buffer if return_buffer else passage

    else:
        ## Subsequent Cases: There is a poem to start with. Generate a similar one with a new topic!

        # yield f"Not Implemented!!!"; return ## <- TODO: Comment this out
                
        ########################################################################
        ## TODO: Invoke the second chain to generate the new rhymes.

        buffer = f"Sure! Here you go!\n\n"
        yield buffer
        
        ## iterate over stream generator for second generation
        chat_gen = chain2.stream({"input" : first_poem, "topic" : message})
        for token in chat_gen:
            buffer += token
            yield buffer if return_buffer else token

        ## END TODO
        ########################################################################

        passage = "\n\nThis is fun! Give me another topic!"
        buffer += passage
        yield buffer if return_buffer else passage

################################################################################
## Below: This is a small-scale simulation of the gradio routine.

def queue_fake_streaming_gradio(chat_stream, history = [], max_questions=3):

    ## Mimic of the gradio initialization routine, where a set of starter messages can be printed off
    for human_msg, agent_msg in history:
        if human_msg: print("\n[ Human ]:", human_msg)
        if agent_msg: print("\n[ Agent ]:", agent_msg)

    ## Mimic of the gradio loop with an initial message from the agent.
    for _ in range(max_questions):
        message = input("\n[ Human ]: ")
        print("\n[ Agent ]: ")
        history_entry = [message, ""]
        for token in chat_stream(message, history, return_buffer=False):
            print(token, end='')
            history_entry[1] += token
        history += [history_entry]
        print("\n")

## history is of format [[User response 0, Bot response 0], ...]
history = [[None, "Let me help you make a poem! What would you like for me to write?"]]

## Simulating the queueing of a streaming gradio interface, using python input
queue_fake_streaming_gradio(
    chat_stream = rhyme_chat2_stream,
    history = history
)


[ Agent ]: Let me help you make a poem! What would you like for me to write?


KeyboardInterrupt: Interrupted by user

In [13]:
## Simple way to initialize history for the ChatInterface
chatbot = gr.Chatbot(value = [[None, "Let me help you make a poem! What would you like for me to write?"]])

## IF USING COLAB: Share=False is faster
gr.ChatInterface(rhyme_chat2_stream, chatbot=chatbot).queue().launch(debug=True, share=True)

/usr/local/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.41.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5fe9d6f21bda489a01.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5fe9d6f21bda489a01.gradio.live



<br>

## **第五部分：[練習]** 使用更深層的 LangChain 整合


這個練習讓您有機會了解一些關於 [**LangServe**](https://python.langchain.com/docs/langserve/) 的範例程式碼。具體來說，我們參考 [**`frontend`**](frontend) 目錄以及 [**`09_langserve.ipynb`**](09_langserve.ipynb) notebook。

-   訪問(Navigate) [**`09_langserve.ipynb`**](09_langserve.ipynb) 並運行提供的腳本(script)以啟動具有多個活躍路由(active routes)的伺服器。
-   完成後，驗證以下 **LangServe `RemoteRunnable`** 是否有效。 **`RemoteRunnable`** 的目標是使將 LangChain 鏈(Chain)託管(Hosting)為 API 端點(Endpoints)變得容易，所以以下只是一個測試以確保它有效。

    -   如果第一次不起作用，可能存在操作順序問題。請隨時嘗試重新啟動 langserve notebook。

**完成這些步驟後，以下類型的連接將可從課程中的任意 notebook 存取：**

In [18]:
from langserve import RemoteRunnable
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

llm = RemoteRunnable("http://0.0.0.0:9012/basic_chat/") | StrOutputParser()
for token in llm.stream("Hello World! How is it going?"):
    print(token, end='')

## Equivalent to the following, assuming you're using the same model
# llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct") | StrOutputParser()
# for token in llm.stream("Hello World! How is it going?"):
#     print(token, end='')

Hello there! I'm doing well, thanks for asking! It's great to meet you. Since I'm just a language model, I don't have feelings or emotions, but I'm always happy to chat with you. How about you? How's your day going so far?

<br>

這個端點(Endpoints)的活躍使用者之一是 `frontend`，它在其 [**`frontend_server.py`**](./frontend/frontend_server.py) 實作中引用了它：

```python
## Necessary Endpoints
chains_dict = {
    'basic' : RemoteRunnable("http://lab:9012/basic_chat/"),
    'retriever' : RemoteRunnable("http://lab:9012/retriever/"),  ## For the final assessment
    'generator' : RemoteRunnable("http://lab:9012/generator/"),  ## For the final assessment
}

basic_chain = chains_dict['basic']

## Retrieval-Augmented Generation Chain

retrieval_chain = (
    {'input' : (lambda x: x)}
    | RunnableAssign(
        {'context' : itemgetter('input') 
        | chains_dict['retriever'] 
        | LongContextReorder().transform_documents
        | docs2str
    })
)

output_chain = RunnableAssign({"output" : chains_dict['generator'] }) | output_puller
rag_chain = retrieval_chain | output_chain
```

因此，部署(Deploying) '/basic_chat' 鏈(Chain)前應該實作前端介面中的 **「基本」** 聊天功能。提醒一下，您可以透過以下生成的連結存取前端：

In [19]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:#76b900;" target="_blank" href='+url+'><h2>< Link To Gradio Frontend ></h2></a>';

<IPython.core.display.Javascript object>



**當您開始進行評量(Assessment)時，您將重新思考(revisiting)這個想法。**




**注意：** 在這種類型的環境中部署架構(Deployment)和依賴於 LangServe API 的這種策略非常非標準，專門為學生提供一些有趣的程式碼來查看。更穩定的配置可以通過最佳化的單函式(function)容器實現，可以在 [**NVIDIA/GenerativeAIExamples GitHub 儲存庫**](https://github.com/NVIDIA/GenerativeAIExamples/tree/main/RAG/notebooks) 中找到。



<br>

## **第六部分：** 總結


這個 notebook 的目標是讓您接觸 LangChain 表達式語言方案(LangChain Expression Language scheme Schemes)，並提供 `gradio` 和 `LangServe` 介面的接觸，用於提供 LLM 功能！在後續的 notebook 中會有更多這方面的內容，但這個 notebook 推動了 LLM Agent 開發中的中級和新興作法(Paradigm)。

<font color="#76b900">**做得很好！**</font>


**下一步：**


1.  **[可選]** 花幾分鐘查看 `frontend` 目錄，了解部署架構(Deployment)配方和底層功能。

2.  **[可選]** 重新訪問(Navigate) notebook 頂部的**「值得思考的問題」部分**，並思考一些可能的答案。

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

